In [3]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
import nltk
from nltk import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# reading in the data
train_data = pd.read_csv("WikiQA-train.tsv", sep="\t")
test_data = pd.read_csv("WikiQA-test.tsv", sep="\t")

Extract the unique questions from the train and test data frames, including the documentID and the DocumentTitle


In [5]:
def get_questions_documenttag(data):
    qd = data[
        ["Question", "QuestionID", "DocumentID", "DocumentTitle"]
    ].drop_duplicates()
    return qd


train_question_doctag = get_questions_documenttag(train_data)
test_question_doctag = get_questions_documenttag(test_data)

In [6]:
# get unique questions
train_questions = train_question_doctag["Question"]
test_questions = test_question_doctag["Question"]

In [7]:
# get the unique document ids
train_docid = train_question_doctag["DocumentID"]
test_docid = test_question_doctag["DocumentID"]

Extract the answers to those questions.


In [8]:
def get_answers(data, questions, documentids):
    answers = []  # list of answers
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q]  # add the document id
        df = data[data["Question"] == question]
        index = df.loc[df["Label"] == 1]["Sentence"].index.values
        if len(index) == 0:  # if no answer found
            answers.append([question, doc_id, "No answer"])
        else:  # if 1 answer found
            answers.append([question, doc_id, df.loc[index[0], "Sentence"]])
    return answers


train_answers = pd.DataFrame(get_answers(train_data, train_questions, train_docid))
test_answers = pd.DataFrame(get_answers(test_data, test_questions, test_docid))

The above get_answers returns train_answers and test_answers which, gives us in the following columns

-   Question
-   Related Document ID
-   Answer (if no answer to that question, return no answer)


In [9]:
def get_documents(data, questions, documentids):  # (done by Finn, tweaked by Dan)
    documents = []
    for q in range(len(questions)):
        question = questions.iloc[q]
        doc_id = documentids.iloc[q]  # add the document id
        df = data[data["Question"] == question]
        sentences = df["Sentence"].tolist()
        for i in range(0, len(sentences) - 1):
            sentences[i] = sentences[i] + " "
        documents.append([doc_id, "".join(sentences)])
    return documents


train_documents = pd.DataFrame(
    get_documents(train_data, train_questions, train_docid)
)  # return the individual document in list
test_documents = pd.DataFrame(
    get_documents(test_data, test_questions, test_docid)
)  # return the individual document in list

The above train_documents and test_documents called from the get_documents gives us in the following columns

-   Document ID
-   Full Document


In [10]:
# renaming all the columns for more standardised access
train_answers.columns = ["Question", "DocumentID", "Answer"]
test_answers.columns = ["Question", "DocumentID", "Answer"]
train_documents.columns = ["DocumentID", "Document"]
test_documents.columns = ["DocumentID", "Document"]

In [11]:
# result is 2117, 2117, 630, 630

len(train_answers), len(train_documents), len(test_answers), len(test_documents)

(2117, 2117, 630, 630)

**Prior to tagging, we should maybe clean the document and answers first:** (stopped here)

Maybe?

-   lowercase (might lose context, but we can use on questions)
-   removing any punctuation or weird symbols (do)
-   removal of stop words? (probably not)

Make sure that the pre-processing is standardised to be the same throughout doc and ans.


In [13]:

# These are just common English contractions. We used it in Lab 5 before!
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", 
                    "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                    "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", 
                    "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", 
                    "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", 
                    "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", 
                    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", 
                    "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                    "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
                    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", 
                    "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                    "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
                    "so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", 
                    "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", 
                    "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", 
                    "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", 
                    "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", 
                    "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", 
                    "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", 
                    "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", 
                    "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                    "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}



In [14]:
def preprocess_lower(text):
    # Lowercase the text for question, answer and documents
    text = text.lower()
    for word, new_word in contraction_dict.items():
            text = text.replace(word, new_word) #dealing with contractions
    pattern = r"[^a-zA-Z0-9\s]"
    cleaned_text = re.sub(pattern, " ", text)
    return cleaned_text


train_answers[["Question", "Answer"]] = train_answers[["Question", "Answer"]].applymap(
    preprocess_lower
)
train_documents["Document"] = train_documents["Document"].apply(preprocess_lower)
test_answers[["Question", "Answer"]] = test_answers[["Question", "Answer"]].applymap(
    preprocess_lower
)
test_documents["Document"] = test_documents["Document"].apply(preprocess_lower)

In [15]:
train_documents

,DocumentID,Document
0,D1,a partly submerged glacier cave on perito more...
1,D2,in physics circular motion is a movement of ...
2,D5,apollo creed is a fictional character from the...
3,D6,in the united states the title of federal jud...
4,D7,the beretta 21a bobcat is a small pocket sized...
...,...,...
2112,D2805,blue mountain state is an american comedy seri...
2113,D2806,apple inc formerly apple computer inc is ...
2114,D2807,section 8 housing in the south bronx section 8...
2115,D2808,restaurants categorized by type and informatio...


In [16]:
def labelling(documents, answers):
    tagged_documents = []
    for q in range(len(answers)):
        tagged_document = []
        qn = answers["Question"].loc[q]
        doc_id = answers["DocumentID"].loc[q]
        content = documents.loc[documents["DocumentID"] == doc_id, "Document"].values[0]
        answer = answers["Answer"].loc[q]

        if answer == "no answer":
            tokens = word_tokenize(content)
            for j in range(len(tokens)):
                tagged_document.append("N")  # none
        else:
            parts = content.partition(answer)
            for j in range(len(parts)):
                tokens = word_tokenize(parts[j])
                if j == 1:
                    tagged_document.append("S")  # start of answer
                    for k in range(len(tokens) - 2):
                        tagged_document.append("I")  # inside of answer
                    tagged_document.append("E")  # end of answer
                else:
                    for k in range(len(tokens)):
                        tagged_document.append("N")  # outside answer
        tagged_documents.append(tagged_document)
    return tagged_documents


train_doc_ans_labels = labelling(train_documents, train_answers)
test_doc_ans_labels = labelling(test_documents, test_answers)

In [17]:
# check if tags are good
def testing_tokens(ind, labels, documents, answers):
    for i, j in zip(labels[ind], word_tokenize(documents["Document"][ind])):
        print([i, j])
    print(answers["Answer"][ind])

testing_tokens(1000, train_doc_ans_labels, train_documents, train_answers)

['N', 'egg']
['N', 'roll']
['N', 'is']
['N', 'a']
['N', 'term']
['N', 'used']
['N', 'for']
['N', 'many']
['N', 'different']
['N', 'foods']
['N', 'around']
['N', 'the']
['N', 'world']
['S', '2']
['I', 'egg']
['I', 'roll']
['I', 'varieties']
['I', 'of']
['I', 'egg']
['I', 'rolls']
['I', 'are']
['I', 'found']
['I', 'in']
['I', 'mainland']
['I', 'china']
['I', 'many']
['I', 'chinese']
['I', 'speaking']
['I', 'regions']
['I', 'of']
['I', 'asia']
['I', 'and']
['I', 'chinese']
['I', 'immigrant']
['I', 'communities']
['I', 'around']
['I', 'the']
['E', 'world']
['N', 'egg']
['N', 'rolls']
['N', 'as']
['N', 'referred']
['N', 'to']
['N', 'in']
['N', 'china']
['N', 'in']
['N', 'guangdong']
['N', 'and']
['N', 'hong']
['N', 'kong']
['N', 'egg']
['N', 'roll']
['N', 'usually']
['N', 'refers']
['N', 'to']
['N', 'biscuit']
['N', 'roll']
['N', 'this']
['N', 'is']
['N', 'a']
['N', 'type']
['N', 'of']
['N', 'biscuit']
['N', 'the']
['N', 'ingredient']
['N', 'included']
['N', 'egg']
['N', 'flour']
['N', 'and

Cleaned Documents: train and test

train_answers - contains the ['Question','DocumentID','Answer']

train_documents - contains the ['DocumentID','Document']

train_doc_ans_labels - contains a list of list of answer tags for each document,


In [18]:
# To prepare the document for word embeddings:
train_doc_ques = pd.DataFrame(
    {"Document": train_documents["Document"], "Question": train_answers["Question"]}
)
test_doc_ques = pd.DataFrame(
    {"Document": test_documents["Document"], "Question": test_answers["Question"]}
)

### Word Embeddings

To use the CBOW model, we need the data in sentences. Extract this from the original dataset, don't use sent_tokenise, will mess with some of the fullstops, we want to maintain structure from above


In [19]:
def word_tokens(data):
    sentence_list = []
    for i in range(len(data)):
        sentence_list.append(word_tokenize(data[i]))
    return sentence_list


train_doc_list = word_tokens(train_doc_ques["Document"])
train_ques_list = word_tokens(train_doc_ques["Question"])
test_doc_list = word_tokens(test_doc_ques["Document"])
test_ques_list = word_tokens(test_doc_ques["Question"])

In [20]:
combined_text = train_doc_list + train_ques_list + test_doc_list + test_ques_list

In [23]:
# model trained, don't have to run this multiple times
wc_cbow_model = Word2Vec(
    sentences=combined_text,
    vector_size=100,
    window=5,
    min_count=1,
    workers=2,
    epochs=30,
)
#wc_cbow_model.save("cbow.model")

To implement QA

1. Word Embeddings, using CBOW
2. Feature Extraction 1 - POS tags
3. Feature Extraction 2 - TF-IDF
4. Feature Extraction 3 - NER


In [21]:
# run this if model in directory 
wc_cbow_model = Word2Vec.load("./cbow.model")

In [22]:
def get_word_embeddings(doc):
    tokenized_doc = word_tokenize(doc)
    embeddings = [wc_cbow_model.wv[word] for word in tokenized_doc]
    return embeddings


train_doc_ques["Doc_Embeddings"] = train_doc_ques["Document"].apply(get_word_embeddings)
train_doc_ques["Q_Embeddings"] = train_doc_ques["Question"].apply(get_word_embeddings)
test_doc_ques["Doc_Embeddings"] = test_doc_ques["Document"].apply(get_word_embeddings)
test_doc_ques["Q_Embeddings"] = test_doc_ques["Question"].apply(get_word_embeddings)

In [23]:
train_doc_ques["Doc_Tokens"] = train_doc_ques["Document"].apply(word_tokenize)
train_doc_ques["Q_Tokens"] = train_doc_ques["Question"].apply(word_tokenize)
test_doc_ques["Doc_Tokens"] = test_doc_ques["Document"].apply(word_tokenize)
test_doc_ques["Q_Tokens"] = test_doc_ques["Question"].apply(word_tokenize)

In [24]:
def find_max_length(column):
    
    max_length = 0
    lns = []
    for i in range(len(column)):
        lns.append(len(column[i]))
        if len(column[i]) > max_length:
            max_length = len(column[i])
    return ("Max Seq Length is {}, Median is {}, Number of lines is {})".format(max_length, np.median(lns), len(lns)))

find_max_length(test_doc_ques["Doc_Embeddings"])
find_max_length(train_doc_ques["Doc_Embeddings"])

'Max Seq Length is 924, Median is 182.0, Number of lines is 2117)'

In [25]:
def check_count(doc):
    count = 0
    for i in range(len(doc)):
        if len(doc["Doc_Embeddings"][i]) != len(doc["Doc_Tokens"][i]):
            count += 1
        elif len(doc["Q_Embeddings"][i]) != len(doc["Q_Tokens"][i]):
            count += 1
        else:
            continue
    return count


check_count(train_doc_ques)  # looks good

0

Note, need to convert the POS tags, NER tags into embeddings. After this, pad the questions and answers to the max question/document length in the combined training and test set.

### PoS Tagging


In [26]:
# Apply the pos tags to the tokens
from nltk.tag import pos_tag

# download the dependency and resource as required
nltk.download("averaged_perceptron_tagger")

train_doc_ques["Doc_POS"] = train_doc_ques["Doc_Tokens"].apply(pos_tag)
train_doc_ques["Q_POS"] = train_doc_ques["Q_Tokens"].apply(pos_tag)
test_doc_ques["Doc_POS"] = test_doc_ques["Doc_Tokens"].apply(pos_tag)
test_doc_ques["Q_POS"] = test_doc_ques["Q_Tokens"].apply(pos_tag)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/daniel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:
# checking the POS tags: # looks ok
train_doc_ques["Q_POS"][100]

[('how', 'WRB'),
 ('many', 'JJ'),
 ('schools', 'NNS'),
 ('are', 'VBP'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('big', 'JJ'),
 ('ten', 'NN')]

In [28]:
# Extract all unique POS Tags
all_pos_tags = (
    train_doc_ques["Doc_POS"].tolist()
    + test_doc_ques["Doc_POS"].tolist()
    + train_doc_ques["Q_POS"].tolist()
    + test_doc_ques["Q_POS"].tolist()
)


def get_unique_pos(data):
    pos_tags = set()
    for item in data:
        for _, pos_tag in item:
            pos_tags.add(pos_tag)

    pos_tag_index = {tag: i for i, tag in enumerate(sorted(pos_tags))}
    return pos_tag_index


pos_iden = get_unique_pos(all_pos_tags)  # list of tags
pos_iden

{'$': 0,
 'CC': 1,
 'CD': 2,
 'DT': 3,
 'EX': 4,
 'FW': 5,
 'IN': 6,
 'JJ': 7,
 'JJR': 8,
 'JJS': 9,
 'MD': 10,
 'NN': 11,
 'NNP': 12,
 'NNPS': 13,
 'NNS': 14,
 'PDT': 15,
 'POS': 16,
 'PRP': 17,
 'PRP$': 18,
 'RB': 19,
 'RBR': 20,
 'RBS': 21,
 'RP': 22,
 'SYM': 23,
 'TO': 24,
 'UH': 25,
 'VB': 26,
 'VBD': 27,
 'VBG': 28,
 'VBN': 29,
 'VBP': 30,
 'VBZ': 31,
 'WDT': 32,
 'WP': 33,
 'WP$': 34,
 'WRB': 35}

### NER Tagging


### Steps to run this:

-   pip install spacy
-   python -m spacy download en_core_web_sm

If loaded for the first time, restart kernel


In [30]:
# nltk using Spacy
# pip install -U spacy
# python -m spacy download en_core_web_sm
import spacy
import en_core_web_sm

# loading pre-trained model of NER
#nlp = en_core_web_sm.load()
#nlp.to_disk('en_core_web_sm')

2023-05-18 18:14:02.660149: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [31]:
nlp = spacy.load('./en_core_web_sm/')

In [33]:
def ner_tagging(texts):
    tagged_texts = []
    for text in texts:
        doc = spacy.tokens.Doc(nlp.vocab, words=text)
        nlp.get_pipe("ner")(doc)
        tagged_texts.append([(token.text, token.ent_type_) for token in doc])
    return tagged_texts

In [34]:
# Will take a while...
train_doc_ques["Doc_NER"] = ner_tagging(train_doc_ques["Doc_Tokens"])
train_doc_ques["Q_NER"] = ner_tagging(train_doc_ques["Q_Tokens"])
test_doc_ques["Doc_NER"] = ner_tagging(test_doc_ques["Doc_Tokens"])
test_doc_ques["Q_NER"] = ner_tagging(test_doc_ques["Q_Tokens"])

In [48]:
train_doc_ques["Q_NER"][12]

[('how', ''),
 ('much', ''),
 ('are', ''),
 ('the', ''),
 ('harry', 'WORK_OF_ART'),
 ('potter', 'WORK_OF_ART'),
 ('movies', 'WORK_OF_ART'),
 ('worth', 'WORK_OF_ART')]

In [35]:
# Similar approach to the POS

# Extract all unique POS Tags
all_ner_tags = (
    train_doc_ques["Doc_NER"].tolist()
    + test_doc_ques["Doc_NER"].tolist()
    + train_doc_ques["Q_NER"].tolist()
    + test_doc_ques["Q_NER"].tolist()
)


def get_unique_ner(data):
    ner_tags = set()
    for item in data:
        for _, ner_tag in item:
            ner_tags.add(ner_tag)

    ner_tag_index = {tag: i for i, tag in enumerate(sorted(ner_tags))}
    return ner_tag_index


ner_iden = get_unique_pos(all_ner_tags)  # list of tags
ner_iden

{'': 0,
 'CARDINAL': 1,
 'DATE': 2,
 'EVENT': 3,
 'FAC': 4,
 'GPE': 5,
 'LANGUAGE': 6,
 'LAW': 7,
 'LOC': 8,
 'MONEY': 9,
 'NORP': 10,
 'ORDINAL': 11,
 'ORG': 12,
 'PERCENT': 13,
 'PERSON': 14,
 'PRODUCT': 15,
 'QUANTITY': 16,
 'TIME': 17,
 'WORK_OF_ART': 18}

In [36]:
# check ohv dims
ner_idx = ner_iden.values()
aa = np.eye(max(ner_idx) + 1)
#aa

### TF-IDF

First, calculate the document frequency of each token in the entire corpus (training documents + testing documents). The result is a dictionary where each token is a key and its value is the document frequency.


In [37]:
def document_frequency(corpus):
    """
    Computes the document frequency for every token in the corpus.
    Returns a dictionary {token: doc_freq, ...}
    """
    document_frequency = {}
    for document in corpus:
        for token in np.unique(document):
            try:
                document_frequency[token] += 1
            except:
                document_frequency[token] = 1
    return document_frequency


train_corpus = (
    train_doc_ques["Doc_Tokens"].tolist() + train_doc_ques["Q_Tokens"].tolist()
)
test_corpus = test_doc_ques["Doc_Tokens"].tolist() + test_doc_ques["Q_Tokens"].tolist()
train_doc_freq = document_frequency(train_corpus)
test_doc_freq = document_frequency(test_corpus)

Now calculate TF-IDF using the document frequency from above.


In [38]:
from collections import Counter
import math


def compute_tf_idf(corpus, doc_frequency):
    """
    Computes the term frequency inverse document frequency for every token in every document in the corpus.
    Returns a list the same shape as the list of tokenized documents except every token is replaced with the tf-idf
    for that token.
    """
    tf_idf = {}
    tf_idf_list = []
    N = len(doc_frequency)
    doc_id = 0
    for document in corpus:
        tf_idf_doc = []
        counter = Counter(document)
        total_num_words = len(document)
        for token in np.unique(document):
            tf = counter[token] / total_num_words
            df = doc_frequency[token]
            idf = math.log(N / (df + 1)) + 1
            tf_idf[doc_id, token] = tf * idf
        for token in document:
            tf_idf_doc.append(tf_idf[doc_id, token])
        tf_idf_list.append(tf_idf_doc)
        doc_id += 1
    return tf_idf_list


train_doc_ques["Doc_TFIDF"] = compute_tf_idf(
    train_doc_ques["Doc_Tokens"].tolist(), train_doc_freq
)
train_doc_ques["Q_TFIDF"] = compute_tf_idf(
    train_doc_ques["Q_Tokens"].tolist(), train_doc_freq
)
test_doc_ques["Doc_TFIDF"] = compute_tf_idf(
    test_doc_ques["Doc_Tokens"].tolist(), test_doc_freq
)
test_doc_ques["Q_TFIDF"] = compute_tf_idf(
    test_doc_ques["Q_Tokens"].tolist(), test_doc_freq
)

In [39]:
test_doc_ques

,Document,Question,Doc_Embeddings,Q_Embeddings,Doc_Tokens,Q_Tokens,Doc_POS,Q_POS,Doc_NER,Q_NER,Doc_TFIDF,Q_TFIDF
0,african immigration to the united states refer...,how african americans were immigrated to the us,"[[-0.16284928, -0.5099784, -2.502221, 0.793150...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[african, immigration, to, the, united, states...","[how, african, americans, were, immigrated, to...","[(african, JJ), (immigration, NN), (to, TO), (...","[(how, WRB), (african, JJ), (americans, NNS), ...","[(african, ORG), (immigration, ORG), (to, ), (...","[(how, ), (african, NORP), (americans, NORP), ...","[0.2444338691504048, 0.1690021701802108, 0.283...","[0.708514287434769, 0.9349595495002982, 0.9907..."
1,a prison from old french prisoun also known...,how large were early jails,"[[0.6947981, -0.006612428, -0.9311606, 0.93038...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[a, prison, from, old, french, prisoun, also, ...","[how, large, were, early, jails]","[(a, DT), (prison, NN), (from, IN), (old, JJ),...","[(how, WRB), (large, JJ), (were, VBD), (early,...","[(a, ), (prison, ), (from, ), (old, ), (french...","[(how, ), (large, ), (were, ), (early, ), (jai...","[0.23734258656531904, 0.15055215160158517, 0.0...","[1.1336228598956304, 1.2987719202959243, 1.145..."
2,a small electrically powered pump a large el...,how a water pump works,"[[0.6947981, -0.006612428, -0.9311606, 0.93038...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[a, small, electrically, powered, pump, a, lar...","[how, a, water, pump, works]","[(a, DT), (small, JJ), (electrically, RB), (po...","[(how, WRB), (a, DT), (water, NN), (pump, NN),...","[(a, ), (small, ), (electrically, ), (powered,...","[(how, ), (a, ), (water, ), (pump, ), (works, )]","[0.1817774325199266, 0.07355137595811154, 0.19...","[1.1336228598956304, 0.8361761895916624, 1.412..."
3,lolita is a 1962 comedy drama film by stanley ...,how old was sue lyon when she made lolita,"[[-0.11471885, 0.26024267, 0.2678885, 0.086137...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[lolita, is, a, 1962, comedy, drama, film, by,...","[how, old, was, sue, lyon, when, she, made, lo...","[(lolita, NN), (is, VBZ), (a, DT), (1962, CD),...","[(how, WRB), (old, JJ), (was, VBD), (sue, NN),...","[(lolita, ), (is, ), (a, ), (1962, DATE), (com...","[(how, ), (old, ), (was, ), (sue, PERSON), (ly...","[0.22677023461895618, 0.03157635832223484, 0.0...","[0.6297904777197946, 0.7657655256778074, 0.528..."
4,each antibody binds to a specific antigen an...,how are antibodies used in,"[[1.1073288, 2.4011667, -1.4736867, 2.3350687,...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[each, antibody, binds, to, a, specific, antig...","[how, are, antibodies, used, in]","[(each, DT), (antibody, NN), (binds, VBZ), (to...","[(how, WRB), (are, VBP), (antibodies, NNS), (u...","[(each, ), (antibody, ), (binds, ), (to, ), (a...","[(how, ), (are, ), (antibodies, ), (used, ), (...","[0.05405354788321593, 0.24067984633049655, 0.0...","[1.1336228598956304, 0.9669842090544979, 1.862..."
...,...,...,...,...,...,...,...,...,...,...,...,...
625,american cuts of beef including the brisket br...,where is the brisket from,"[[1.6384165, 0.26733622, -0.88549775, 2.357230...","[[-1.611623, 1.7158104, 0.45376357, 1.2674458,...","[american, cuts, of, beef, including, the, bri...","[where, is, the, brisket, from]","[(american, JJ), (cuts, NNS), (of, IN), (beef,...","[(where, WRB), (is, VBZ), (the, DT), (brisket,...","[(american, NORP), (cuts, ), (of, ), (beef, ),...","[(where, ), (is, ), (the, ), (brisket, ), (fro...","[0.04467110190076254, 0.15609658426345493, 0.2...","[1.1855735192809844, 0.8020395013847651, 0.792..."
626,the arm architecture describes a family of ris...,what is arm chipset,"[[-1.8290923, -0.30621898, 0.3005725, 0.443662...","[[1.906984, 3.2368956, -2.5320153, -2.3044674,...","[the, arm, architecture, describes, a, family,...","[what, is, arm, chipset]","[(the, DT), (arm, NN), (archite

In [40]:
train_doc_ques

,Document,Question,Doc_Embeddings,Q_Embeddings,Doc_Tokens,Q_Tokens,Doc_POS,Q_POS,Doc_NER,Q_NER,Doc_TFIDF,Q_TFIDF
0,a partly submerged glacier cave on perito more...,how are glacier caves formed,"[[0.6947981, -0.006612428, -0.9311606, 0.93038...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[a, partly, submerged, glacier, cave, on, peri...","[how, are, glacier, caves, formed]","[(a, DT), (partly, RB), (submerged, VBN), (gla...","[(how, WRB), (are, VBP), (glacier, JJ), (caves...","[(a, ), (partly, ), (submerged, ), (glacier, )...","[(how, ), (are, ), (glacier, ), (caves, ), (fo...","[0.24677746860673927, 0.1451147752938821, 0.16...","[1.034209132108837, 0.8495245060136498, 1.9896..."
1,in physics circular motion is a movement of ...,how are the directions of the velocity and for...,"[[-0.3025114, 0.803383, 0.88826, 0.36154857, 1...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[in, physics, circular, motion, is, a, movemen...","[how, are, the, directions, of, the, velocity,...","[(in, IN), (physics, NNS), (circular, JJ), (mo...","[(how, WRB), (are, VBP), (the, DT), (direction...","[(in, ), (physics, ), (circular, ), (motion, )...","[(how, ), (are, ), (the, ), (directions, ), (o...","[0.10157954319378303, 0.047977208560696934, 0....","[0.34473637736961227, 0.28317483533788324, 0.4..."
2,apollo creed is a fictional character from the...,how did apollo creed die,"[[-0.08092001, 0.23716491, 0.7449493, 0.191466...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[apollo, creed, is, a, fictional, character, f...","[how, did, apollo, creed, die]","[(apollo, NNS), (creed, VBP), (is, VBZ), (a, D...","[(how, WRB), (did, VBD), (apollo, VB), (creed,...","[(apollo, ORG), (creed, ), (is, ), (a, ), (fic...","[(how, ), (did, ), (apollo, ORG), (creed, ), (...","[0.15061458188394025, 0.28752101617726733, 0.0...","[1.034209132108837, 1.1532031921635457, 1.7872..."
3,in the united states the title of federal jud...,how long is the term for federal judges,"[[-0.3025114, 0.803383, 0.88826, 0.36154857, 1...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[in, the, united, states, the, title, of, fede...","[how, long, is, the, term, for, federal, judges]","[(in, IN), (the, DT), (united, JJ), (states, V...","[(how, WRB), (long, JJ), (is, VBZ), (the, DT),...","[(in, ), (the, GPE), (united, GPE), (states, G...","[(how, ), (long, ), (is, ), (the, ), (term, ),...","[0.08828641778817685, 0.32044981985432075, 0.1...","[0.646380707568023, 0.7777965845277852, 0.4296..."
4,the beretta 21a bobcat is a small pocket sized...,how a beretta model 21 pistols magazines works,"[[-1.8290923, -0.30621898, 0.3005725, 0.443662...","[[0.08589837, 2.4839535, -2.2136166, 3.3471255...","[the, beretta, 21a, bobcat, is, a, small, pock...","[how, a, beretta, model, 21, pistols, magazine...","[(the, DT), (beretta, NN), (21a, CD), (bobcat,...","[(how, WRB), (a, DT), (beretta, NN), (model, N...","[(the, ), (beretta, ), (21a, ), (bobcat, ), (i...","[(how, ), (a, ), (beretta, PRODUCT), (model, )...","[0.21160137561032594, 0.8290055086215778, 0.22...","[0.646380707568023, 0.4549959577436755, 1.2435..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2112,blue mountain state is an american comedy seri...,where was blue mountain state filmed at,"[[-0.2716396, 2.832203, 2.2265713, 1.0802964, ...","[[-1.611623, 1.7158104, 0.45376357, 1.2674458,...","[blue, mountain, state, is, an, american, come...","[where, was, blue, mountain, state, filmed, at]","[(blue, JJ), (mountain, NN), (state, NN), (is,...","[(where, WRB), (was, VBD), (blue, JJ), (mounta...","[(blue, LOC), (mountain, LOC), (state, ), (is,...","[(where, ), (was, ), (blue, ), (mountain, ), (...","[0.29475406599874054, 0.3823486411848812, 0.22...","[0.7503059363146255, 0.5906688741268076, 1.063..."
2113,apple inc formerly apple computer inc is ...,when was apple computer founded,"[[0.24397807, 0.11627979, 0.98766154, -1.33965...","[[-3.38445, 1.818213, -0.19103132, -0.26652685...","[apple, inc, formerly,

In [41]:
def one_hot_vectorize(
    pos_tagger, ner_tagger, data
):  # pass in the unique dict for ner or pos
    pos_idx = pos_tagger.values()
    pos_ohv = np.eye(max(pos_idx) + 1)  # create the ohv
    ner_idx = ner_tagger.values()
    ner_ohv = np.eye(max(ner_idx) + 1)

    dpos_full_ohv, dner_full_ohv = [], []  # lists to append to
    qpos_full_ohv, qner_full_ohv = [], []  # lists to append to

    for item in data["Doc_POS"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            pos_index_iden = pos_tagger[tag]
            sent_ohv.append(pos_ohv[pos_index_iden])
        dpos_full_ohv.append(sent_ohv)

    for item in data["Q_POS"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            pos_index_iden = pos_tagger[tag]
            sent_ohv.append(pos_ohv[pos_index_iden])
        qpos_full_ohv.append(sent_ohv)

    for item in data["Doc_NER"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            ner_index_iden = ner_tagger[tag]
            sent_ohv.append(ner_ohv[ner_index_iden])
        dner_full_ohv.append(sent_ohv)

    for item in data["Q_NER"]:
        sent_ohv = []
        for word in item:
            tag = word[1]
            ner_index_iden = ner_tagger[tag]
            sent_ohv.append(ner_ohv[ner_index_iden])
        qner_full_ohv.append(sent_ohv)

    return (dpos_full_ohv, qpos_full_ohv, dner_full_ohv, qner_full_ohv)

In [42]:
# get the ohv for doc
(
    train_doc_pos_ohv,
    train_q_pos_ohv,
    train_doc_ner_ohv,
    train_q_ner_ohv,
) = one_hot_vectorize(pos_iden, ner_iden, train_doc_ques)
test_doc_pos_ohv, test_q_pos_ohv, test_doc_ner_ohv, test_q_ner_ohv = one_hot_vectorize(
    pos_iden, ner_iden, test_doc_ques
)

In [43]:
# reduce the dataframe to just tokens and embeddings:
doc_emb_train = train_doc_ques[["Doc_Tokens", "Doc_Embeddings", "Doc_TFIDF"]]
doc_pos_ner = pd.DataFrame({"Doc_POS": train_doc_pos_ohv, "Doc_NER": train_doc_ner_ohv})
doc_emb_train = pd.concat([doc_emb_train, doc_pos_ner], axis=1)

q_emb_train = train_doc_ques[["Q_Tokens", "Q_Embeddings", "Q_TFIDF"]]
q_pos_ner = pd.DataFrame({"Q_POS": train_q_pos_ohv, "Q_NER": train_q_ner_ohv})
q_emb_train = pd.concat([q_emb_train, q_pos_ner], axis=1)

In [44]:
doc_emb_test = test_doc_ques[["Doc_Tokens", "Doc_Embeddings", "Doc_TFIDF"]]
doc_pos_ner = pd.DataFrame({"Doc_POS": test_doc_pos_ohv, "Doc_NER": test_doc_ner_ohv})
doc_emb_test = pd.concat([doc_emb_test, doc_pos_ner], axis=1)

q_emb_test = test_doc_ques[["Q_Tokens", "Q_Embeddings", "Q_TFIDF"]]
q_pos_ner = pd.DataFrame({"Q_POS": test_q_pos_ohv, "Q_NER": test_q_ner_ohv})
q_emb_test = pd.concat([q_emb_test, q_pos_ner], axis=1)

### Word Embeddings (Doc and Qn)

The embeddings of the questions and answers of the train and test set can be found here:

-   Train Document - doc_emb_train
-   Train Q - q_emb_train
-   Test Document - doc_emb_test
-   Test Q - q_emb_test

The max_document size is 1675 and max_question size is 23.


In [45]:
doc_emb_train

,Doc_Tokens,Doc_Embeddings,Doc_TFIDF,Doc_POS,Doc_NER
0,"[a, partly, submerged, glacier, cave, on, peri...","[[0.6947981, -0.006612428, -0.9311606, 0.93038...","[0.24677746860673927, 0.1451147752938821, 0.16...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,"[in, physics, circular, motion, is, a, movemen...","[[-0.3025114, 0.803383, 0.88826, 0.36154857, 1...","[0.10157954319378303, 0.047977208560696934, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,"[apollo, creed, is, a, fictional, character, f...","[[-0.08092001, 0.23716491, 0.7449493, 0.191466...","[0.15061458188394025, 0.28752101617726733, 0.0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,"[in, the, united, states, the, title, of, fede...","[[-0.3025114, 0.803383, 0.88826, 0.36154857, 1...","[0.08828641778817685, 0.32044981985432075, 0.1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,"[the, beretta, 21a, bobcat, is, a, small, pock...","[[-1.8290923, -0.30621898, 0.3005725, 0.443662...","[0.21160137561032594, 0.8290055086215778, 0.22...","[[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
...,...,...,...,...,...
2112,"[blue, mountain, state, is, an, american, come...","[[-0.2716396, 2.832203, 2.2265713, 1.0802964, ...","[0.29475406599874054, 0.3823486411848812, 0.22...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
2113,"[apple, inc, formerly, apple, computer, inc, i...","[[0.24397807, 0.11627979, 0.98766154, -1.33965...","[0.2615737575545419, 0.08943278683987749, 0.02...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2114,"[section, 8, housing, in, the, south, bronx, s...","[[0.83727205, 0.588537, -0.8883795, 1.2417848,...","[0.19207290739926391, 0.16089672153840806, 0.2...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0,..."
2115,"[restaurants, categorized, by, type, and, info...","[[0.62694025, 0.13019273, -0.10285008, 1.67548...","[0.876941110711729, 0.9254918922898989, 0.4042...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [32]:
doc_emb_train['Doc_NER'][0]

NameError: name 'doc_emb_train' is not defined

In [46]:
def full_array(data, data_type="Document"):
    num_vec_length = 156
    max_doc = 1675
    max_qn = 23
    zero_vec = np.zeros(156)

    if data_type == "Document":
        full_vec = []  # create a list for list of list for document
        for dat in range(len(data)):  # go through each line
            doc_ques = data.loc[dat]  # document data
            v = []  # create list to each word
            for j in range(len(doc_ques.iloc[0])):
                vn = []  # list of concat word embeddings
                vn.append(doc_ques.iloc[1][j].tolist()) #Word2Vec
                vn.append(doc_ques.iloc[2][j]) # TF-IDF
                vn.append(doc_ques.iloc[3][j].tolist()) # POS
                vn.append(doc_ques.iloc[4][j].tolist()) # NER
                flatten = [
                    item
                    for sublist in vn
                    for item in (sublist if isinstance(sublist, list) else [sublist])
                ]
                v.append(flatten)
            while len(v) < max_doc:
                v.append(zero_vec)
            full_vec.append(v)

    if data_type == "Question":
        full_vec = []  # create a list for list of list for document
        for dat in range(len(data)):  # go through each line
            doc_ques = data.loc[dat]  # document data
            v = []  # create list to each word
            for j in range(len(doc_ques.iloc[0])):
                vn = []  # list of concat word embeddings
                vn.append(doc_ques.iloc[1][j].tolist()) #Word2Vec
                vn.append(doc_ques.iloc[2][j]) # TF-IDF
                vn.append(doc_ques.iloc[3][j].tolist()) #POS
                vn.append(doc_ques.iloc[4][j].tolist()) #NER
                flatten = [
                    item
                    for sublist in vn
                    for item in (sublist if isinstance(sublist, list) else [sublist])
                ]
                v.append(flatten)
            while len(v) < max_qn:
                v.append(zero_vec)
            full_vec.append(v)
    return full_vec

In [47]:
# Training/Test Documents to pass in, takes about a min
final_doc_train = np.stack(full_array(doc_emb_train, data_type="Document"))
final_doc_test = np.stack(full_array(doc_emb_test, data_type="Document"))

In [48]:
# Training/Test Questions to pass in, takes about a few seconds
final_qn_train = np.stack(full_array(q_emb_train, data_type="Question"))
final_qn_test = np.stack(full_array(q_emb_test, data_type="Question"))

In [50]:
def convert_labels(labels):
    check = []
    for i in labels:
        if len(i) < 1675:
            while len(i) < 1675:
                i.append('N')
            check.append(i)
        else:
            check.append(i)
    return check
tr_labels = np.array(convert_labels(train_doc_ans_labels))
ts_labels = np.array(convert_labels(test_doc_ans_labels))

Earlier, we found that the sequence length of the documents can get quite large, which might introduce a lot of noise into the model with paddings. One alternative to reduce this noise is to perhaps truncate the sequences down to just over the median for the documents. We also need to do this for the outputs.

In [51]:
find_max_length(test_doc_ques["Doc_Embeddings"]),find_max_length(train_doc_ques["Doc_Embeddings"]), find_max_length(test_doc_ques["Q_Embeddings"]), find_max_length(train_doc_ques["Q_Embeddings"])

('Max Seq Length is 1675, Median is 181.0, Number of lines is 630)',
 'Max Seq Length is 924, Median is 182.0, Number of lines is 2117)',
 'Max Seq Length is 19, Median is 7.0, Number of lines is 630)',
 'Max Seq Length is 23, Median is 7.0, Number of lines is 2117)')

In [52]:
# truncate the embeddings and the labels to 200

def truncate(data, labels, max_seq=200):
    data = data[:, :max_seq, :]
    labels = labels[:, :max_seq]
    return data, labels

final_doc_train, tr_labels = truncate(final_doc_train, tr_labels)
final_doc_test, ts_labels = truncate(final_doc_test, ts_labels)

In [ ]:
# final prepared documents are found here:

#final_doc_train, tr_labels - doc embeddings and output labels for training
#final_doc_test, ts_labels - doc embeddings and output labels for testing
#final_qn_train - question embeddings for training
#final_qn_test - question embeddings for testing

### Converting into Tensors:


In [53]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
# takes a min
tf_final_doc_train = torch.tensor(final_doc_train, device=device).to(dtype=torch.float32)
tf_final_doc_test = torch.tensor(final_doc_test, device=device).to(dtype=torch.float32)
tf_final_qn_train = torch.tensor(final_qn_train, device=device).to(dtype=torch.float32)
tf_final_qn_test = torch.tensor(final_qn_test, device=device).to(dtype=torch.float32)

In [55]:
# check dimensions
print(tf_final_doc_train.shape)
print(tf_final_doc_test.shape)
print(tf_final_qn_train.shape)
print(tf_final_qn_test.shape)

torch.Size([2117, 200, 156])
torch.Size([630, 200, 156])
torch.Size([2117, 23, 156])
torch.Size([630, 23, 156])


In [64]:
# save the tensors into wd
torch.save(tf_final_doc_train, 'tensor.doc_train')
torch.save(tf_final_doc_test, 'tensor.doc_test')
torch.save(tf_final_qn_train, 'tensor.qn_train')
torch.save(tf_final_qn_test, 'tensor.qn_test')

**Input Embedding Ablation Study**

In the model input embedding Ablation study, we are given 3 variations of input embeddings to test. We will test 3 options:

1. Word2Vec only # 100 dims
2. Word2Vec + Tf-IDF # 101 dims
3. Word2Vec + all features (TF-IDF, POS, NER) # 156 dims

Since we are using tensors, we can use tensor slicing to take out the relevant features.
Our tensor of embeddings are built as follows (w2v, TF-IDF, POS, NER)

In [66]:
def convert_tensors(tf_doc_train, tf_doc_test, tf_qn_train, tf_qn_test, option=3):
    if option == 3:
        return tf_doc_train, tf_doc_test, tf_qn_train, tf_qn_test
    elif option == 1:
        tf_doc_train = tf_doc_train[:, :, :100]
        tf_doc_test = tf_doc_test[:, :, :100]
        tf_qn_train = tf_qn_train[:, :, :100]
        tf_qn_test = tf_qn_test[:, :, :100]
        return tf_doc_train, tf_doc_test, tf_qn_train, tf_qn_test
    elif option == 2:
        tf_doc_train = tf_doc_train[:, :, :101]
        tf_doc_test = tf_doc_test[:, :, :101]
        tf_qn_train = tf_qn_train[:, :, :101]
        tf_qn_test = tf_qn_test[:, :, :101]
        return tf_doc_train, tf_doc_test, tf_qn_train, tf_qn_test

In [68]:
# change option to see size
(convert_tensors(tf_final_doc_train, tf_final_doc_test, tf_final_qn_train, tf_final_qn_test, 2)[0]).shape

torch.Size([2117, 200, 101])

Our answer should perhaps also be in the form of (1x1675) list containing ['N','S','I','E']

Additionally, the labels should be one hot vectorised, as they are cateogorical.


In [ ]:
np.save('tr_labels.npy', tr_labels)
np.save('ts_labels.npy', ts_labels)

In [ ]:
def one_hot_encode_labels(labels):
    # Create a dictionary that maps each label to a unique integer
    label_to_int = {'N': 0, 'S': 1, 'I': 2, 'E': 3}

    # Map the labels to integers
    int_labels = [[label_to_int[label] for label in sequence] for sequence in labels]

    # Create an identity matrix of size 4 (since there are 4 labels)
    identity = np.eye(4)

    # Use the integer labels as indices to select rows from the identity matrix
    one_hot_labels = [identity[sequence] for sequence in int_labels]
    return one_hot_labels

tr_encoded = one_hot_encode_labels(tr_labels)
tst_encoded = one_hot_encode_labels(ts_labels)

### Model

In [ ]:
# just testing
import torch
import torch.nn as nn

# Bi-LSTM for Document Portion

class Document_LSTM(nn.Module):
    def __init__(self, hidden_size, embedding):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = embedding
        self.lstm = nn.LSTM(hidden_size, hidden_size, ) 
        
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden) 
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)    

class AttnDecoderRNN(nn.Module):
    ATTN_TYPE_DOT_PRODUCT = "Dot Product"
    ATTN_TYPE_SCALE_DOT_PRODUCT = "Scale Dot Product"

    def __init__(self, hidden_size, output_size, embedding, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

    def cal_attention(self, hidden, encoder_hiddens, method):
        # Dot Product Attention
        if method == AttnDecoderRNN.ATTN_TYPE_DOT_PRODUCT:
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)
        # Scaled Dot Product Attention
        elif method == AttnDecoderRNN.ATTN_TYPE_SCALE_DOT_PRODUCT:
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0))/np.sqrt(hidden_size),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)




In [ ]:
from torch import Tensor
import torch.nn as nn
from typing import Literal, Dict, Type, Union
from enum import Enum


class NNType(Enum):
    RNN = "rnn"
    LSTM = "lstm"
    GRU = "gru"

    def __str__(self):
        return self.value


NN_MAP: Dict[NNType, Type[Union[nn.RNN, nn.LSTM, nn.GRU]]] = {
    NNType.RNN: nn.RNN,
    NNType.LSTM: nn.LSTM,
    NNType.GRU: nn.GRU,
}


class EncoderBiRNN(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        embedding: nn.Embedding,
        nn_type: Literal["rnn", "lstm", "gru"] = "rnn",
        num_layers=1,
    ):
        super(EncoderBiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = embedding
        self.nn_type = NNType(nn_type)
        self.nn = NN_MAP[self.nn_type](
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=True,
        )

    def forward(self, input: nn.Embedding, hidden: Tensor):
        embedded: Tensor = self.embedding(input).view(1, 1, -1)
        output: Tensor
        output, hidden = self.nn(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return (
            torch.zeros(2, 1, self.hidden_size, device=device)
            if self.nn_type != NNType.LSTM
            else (
                torch.zeros(2, 1, self.hidden_size, device=device),
                torch.zeros(2, 1, self.hidden_size, device=device),
            )
        )

In [ ]:
import torch.nn.functional as F


class AttentionMethod(Enum):
    DOT_PRODUCT = "dot_product"
    SCALE_DOT_PRODUCT = "scale_dot_product"
    COSINE_SIMILARITY = "cosine_similarity"

    def __str__(self):
        return self.value


class DecoderBiRNN(nn.Module):
    def __init__(
        self,
        hidden_size: int,
        output_size: int,
        embedding: nn.Embedding,
        max_length: int,
        nn_type: Literal["rnn", "lstm", "gru"] = "rnn",
        num_layers=1,
        dropout_p=0.1,
        attention_method: Literal[
            "dot_product",
            "scale_dot_product",
            "cosine_similarity",
        ] = "dot_product",
    ):
        super(DecoderBiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.embedding = embedding
        self.dropout = nn.Dropout(self.dropout_p)
        self.nn_type = NNType(nn_type)
        self.attention_method = AttentionMethod(attention_method)
        self.nn = NN_MAP[self.nn_type](
            hidden_size,
            hidden_size,
            num_layers=num_layers,
            bidirectional=True,
        )
        self.out = nn.Linear(self.hidden_size * 4, self.output_size)

    def cal_attention(self, hidden: Tensor, encoder_hiddens: Tensor):
        if self.attention_method == AttentionMethod.DOT_PRODUCT:
            if self.nn_type == NNType.LSTM:  # For BiLSTM
                energy = torch.bmm(hidden[0], encoder_hiddens.T.repeat(2, 1, 1))
                attn_weights = F.softmax(energy, dim=-1)
                attn_output = torch.bmm(attn_weights, encoder_hiddens.repeat(2, 1, 1))
                concat_output = torch.cat(
                    (attn_output[0], hidden[0][0], attn_output[1], hidden[0][1]), 1
                )
            else:  # For BiRNN & BiGRU
                energy = torch.bmm(hidden, encoder_hiddens.T.repeat(2, 1, 1))
                attn_weights = F.softmax(energy, dim=-1)
                attn_output = torch.bmm(attn_weights, encoder_hiddens.repeat(2, 1, 1))
                concat_output = torch.cat(
                    (attn_output[0], hidden[0], attn_output[1], hidden[1]), 1
                )

        elif self.attention_method == AttentionMethod.COSINE_SIMILARITY:
            if self.nn_type == NNType.LSTM:  # For LSTM
                cosine_similarity = nn.CosineSimilarity(dim=-1)
                h_n, c_n = hidden
                # h_n_reshaped = h_n.mean(dim=0, keepdim=True)
                attn_weights_f = F.softmax(
                    cosine_similarity(h_n[0].unsqueeze(0), encoder_hiddens), dim=-1
                )
                attn_output_f = torch.bmm(
                    attn_weights_f.unsqueeze(0), encoder_hiddens.unsqueeze(0)
                )
                attn_weights_b = F.softmax(
                    cosine_similarity(h_n[1].unsqueeze(0), encoder_hiddens), dim=-1
                )
                attn_output_b = torch.bmm(
                    attn_weights_b.unsqueeze(0), encoder_hiddens.unsqueeze(0)
                )
                concat_output = torch.cat(
                    (
                        attn_output_f[0],
                        h_n[0],
                        attn_output_b[0],
                        h_n[1],
                    ),
                    1,
                )

            else:  # For RNN & GRU
                cosine_similarity = nn.CosineSimilarity(dim=-1)
                # hidden_reshaped = hidden.mean(dim=0, keepdim=True)
                # print(hidden_reshaped.shape)
                attn_weights_f = F.softmax(
                    cosine_similarity(hidden[0].unsqueeze(0), encoder_hiddens), dim=-1
                )
                attn_output_f = torch.bmm(
                    attn_weights_f.unsqueeze(0), encoder_hiddens.unsqueeze(0)
                )
                attn_weights_b = F.softmax(
                    cosine_similarity(hidden[1].unsqueeze(0), encoder_hiddens), dim=-1
                )
                attn_output_b = torch.bmm(
                    attn_weights_b.unsqueeze(0), encoder_hiddens.unsqueeze(0)
                )
                concat_output = torch.cat(
                    (
                        attn_output_f[0],
                        hidden[0],
                        attn_output_b[0],
                        hidden[1],
                    ),
                    1,
                )
        else:
            if self.nn_type == NNType.LSTM:  # For LSTM
                energy = torch.bmm(
                    hidden[0], encoder_hiddens.T.repeat(2, 1, 1)
                ) / np.sqrt(self.hidden_size)
                attn_weights = F.softmax(energy, dim=-1)
                attn_output = torch.bmm(attn_weights, encoder_hiddens.repeat(2, 1, 1))
                concat_output = torch.cat(
                    (attn_output[0], hidden[0][0], attn_output[1], hidden[0][1]), 1
                )
            else:  # For RNN & GRU
                energy = torch.bmm(hidden, encoder_hiddens.T.repeat(2, 1, 1)) / np.sqrt(
                    self.hidden_size
                )
                attn_weights = F.softmax(energy, dim=-1)
                attn_output = torch.bmm(attn_weights, encoder_hiddens.repeat(2, 1, 1))
                concat_output = torch.cat(
                    (attn_output[0], hidden[0], attn_output[1], hidden[1]), 1
                )
        return concat_output

    def forward(self, input: nn.Embedding, hidden: Tensor, encoder_hiddens: Tensor):
        embedded: Tensor = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        _, hidden = self.nn(embedded, hidden)
        concat_output = self.cal_attention(hidden, encoder_hiddens)
        output = F.log_softmax(self.out(concat_output), dim=1)
        return output, hidden

    def initHidden(self):
        return (
            torch.zeros(2, 1, self.hidden_size, device=device)
            if self.nn_type != NNType.LSTM
            else (
                torch.zeros(2, 1, self.hidden_size, device=device),
                torch.zeros(2, 1, self.hidden_size, device=device),
            )
        )

### Training


In [ ]:
MAX_DOC_LENGTH = 1675  # Max doc length
MAX_QN_LENGTH = 23  # Max question length

In [ ]:
def train_test(
    input_tensor,
    target_tensor,
    encoder,
    decoder,
    encoder_optimizer,
    decoder_optimizer,
    criterion,
):
    # Initialize the hidden state of the encoder
    encoder_hidden = encoder.initHidden()

    # Set the gradients of the optimizers to zero
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Get the length of the input and target tensors
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # Initialize the loss to zero
    loss = 0

    # Iterate over the length of the input tensor
    for ei in range(input_length):
        # Pass each element of the input tensor through the encoder
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

    # Set the initial input to the decoder as the first element of the target tensor
    decoder_input = target_tensor[0]
    # Set the initial hidden state of the decoder as the final hidden state of the encoder
    decoder_hidden = encoder_hidden

    # Iterate over the length of the target tensor
    for di in range(target_length):
        # Pass each element of the target tensor through the decoder
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        # Calculate and accumulate the loss by comparing the output of the decoder to the target tensor
        loss += criterion(decoder_output, target_tensor[di])
        # Set the next input to the decoder as the current element of the target tensor
        decoder_input = target_tensor[di]

    # Compute gradients using backpropagation
    loss.backward()

    # Update weights using optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()

    # Return average loss per element in target sequence
    return loss.item() / target_length

In [ ]:
def train(
    input_tensor,
    target_tensor,
    documentRNN,
    questionRNN,
    documentRNN_optimizer,
    questionRNN_optimizer,
    criterion,
    max_doc_length=MAX_DOC_LENGTH,
    max_qn_length=MAX_QN_LENGTH,
    nn_type="rnn",
):
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # it is for storing the hidden states of input sequence later, which will be used for calculating the attention during the decoding process
    documentRNN_hiddens = torch.zeros(
        max_doc_length, documentRNN.hidden_size * 2, device=device
    )

    # zero-initialize an initial hidden state
    documentRNN_hidden = documentRNN.initHidden()
    questionRNN_hidden = questionRNN.initHidden()
    loss = 0
    documentRNN_optimizer.zero_grad()
    questionRNN_optimizer.zero_grad()

    # Feed the input_tensor into the encoder we defined
    for i in range(input_length):
        documentRNN_output, documentRNN_hidden = documentRNN(
            input_tensor[i], documentRNN_hidden
        )
        documentRNN_hiddens[i] = (
            documentRNN_hidden[0][0, 0]
            if nn_type == "lstm"
            else documentRNN_hidden[0, 0]
        )

    # Set the initial input to the decoder as the first element of the target tensor
    questionRNN_input = target_tensor[0]

    # Teacher forcing: Feed the target as the next input
    for i in range(target_length):
        questionRNN_output, questionRNN_hidden = questionRNN(
            questionRNN_input, questionRNN_hidden, documentRNN_hiddens
        )
        loss += criterion(questionRNN_output, target_tensor[i])
        questionRNN_input = target_tensor[i]

    loss.backward()

    documentRNN_optimizer.step()
    questionRNN_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


# Helper functions for training
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return "%s (- %s)" % (asMinutes(s), asMinutes(rs))

In [ ]:
import random
from torch import optim


def trainIters(
    documentRNN,
    questionRNN,
    n_iters,
    print_every=1000,
    plot_every=100,
    learning_rate=0.01,
    nn_type="rnn",
):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    documentRNN_optimizer = optim.AdamW(documentRNN.parameters(), lr=learning_rate)
    questionRNN_optimizer = optim.AdamW(questionRNN.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()
    for iter in range(1, n_iters + 1):
        random_choice_ix = random.choice(
            range(n_data)
        )  # Get a random index within the scope of input data
        input_index_r = [[ind] for ind in input_index[random_choice_ix]]
        target_index_r = [[ind] for ind in target_index[random_choice_ix]]

        input_tensor = torch.LongTensor(input_index_r).to(device)
        target_tensor = torch.LongTensor(target_index_r).to(device)

        loss = train(
            input_tensor,
            target_tensor,
            documentRNN,
            questionRNN,
            documentRNN_optimizer,
            questionRNN_optimizer,
            criterion,
            nn_type=nn_type,
        )
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print(
                "%s (%d %d%%) %.4f"
                % (
                    timeSince(start, iter / n_iters),
                    iter,
                    iter / n_iters * 100,
                    print_loss_avg,
                )
            )

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_sent = pre_process([sentence])[0]
        intput_index = [word_to_ix[word] for word in input_sent]
        input_tensor = torch.LongTensor([[ind] for ind in intput_index]).to(device)

        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            # encoder_hiddens[ei] += encoder_hidden[0, 0]
            encoder_hiddens[ei] = encoder_hidden[0][0, 0]  # LSTM

        decoder_input = torch.tensor([[word_to_ix["<BOS>"]]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_hiddens
            )
            topv, topi = decoder_output.data.topk(
                1
            )  # simply adopt the predicted tag with the highest probabiity
            if (
                topi.item() == word_to_ix["<EOS>"]
            ):  # if <EOS> is generated, stop the generation
                decoded_words.append("<EOS>")
                break
            else:
                decoded_words.append(
                    word_list[topi.item()]
                )  # get the predicted word based on the index
            # use the predicted output as the input for the next time step generation
            decoder_input = topi.squeeze().detach()

        return decoded_words

NameError: name 'MAX_LENGTH' is not defined